In [53]:
import os

import tensorflow as tf
tf.config.experimental_run_functions_eagerly(True)

import h5py
import numpy as np

tf.test.gpu_device_name()

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


'/device:GPU:0'

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Loading datasets

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import preprocess_input

In [4]:
def load_dataset_h5(path, dataset_name):
    with h5py.File(path, "r") as hf:
        print(hf.keys())
        X = hf[dataset_name][:]
        hf.close()
    return X

In [5]:
path = '/content/gdrive/MyDrive/masterDB/train.h5'
path_val = '/content/gdrive/MyDrive/val.h5'

In [50]:
X_train = load_dataset_h5(path, 'X_train')[:4000]
y_train = load_dataset_h5(path, 'y_train')[:4000].reshape((-1,1))

<KeysViewHDF5 ['X_train', 'y_train']>


In [51]:
X_val = load_dataset_h5(path_val, 'X_val')[:800]
y_val = load_dataset_h5(path_val, 'y_val')[:800].reshape((-1,1))

<KeysViewHDF5 ['X_val', 'y_val']>


In [36]:
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.utils import to_categorical


In [37]:
X_train= preprocess_input(X_train)
X_val = preprocess_input(X_val)

# y_train = to_categorical(y_train)
# y_val = to_categorical(y_val)

In [38]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(32)


# Create model

In [16]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, AveragePooling2D

In [47]:
base_model = Xception(include_top=False, weights=None, classes=2)
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])


In [48]:
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

In [ ]:
# results = model.fit(train_dataset, validation_data=test_dataset, epochs=10)
results = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/10
125/125 [==============================] - 265s 2s/step - loss: 0.5820 - accuracy: 0.7020 - val_loss: 0.6964 - val_accuracy: 0.5050
Epoch 2/10
125/125 [==============================] - 265s 2s/step - loss: 0.3733 - accuracy: 0.8342 - val_loss: 0.7119 - val_accuracy: 0.5050
Epoch 3/10
125/125 [==============================] - 264s 2s/step - loss: 0.2833 - accuracy: 0.8855 - val_loss: 1.2065 - val_accuracy: 0.5050
Epoch 4/10
125/125 [==============================] - 265s 2s/step - loss: 0.2116 - accuracy: 0.9140 - val_loss: 8.2825 - val_accuracy: 0.5050
Epoch 5/10
125/125 [==============================] - 265s 2s/step - loss: 0.1926 - accuracy: 0.9215 - val_loss: 131.7789 - val_accuracy: 0.5050
Epoch 6/10
125/125 [==============================] - 265s 2s/step - loss: 0.1439 - accuracy: 0.9423 - val_loss: 15.2843 - val_accuracy: 0.5050
Epoch 7/10
125/125 [==============================] - 265s 2s/step - loss: 0.0932 - accuracy: 0.9632 - val_loss: 767.5129 - val_accuracy: 0

In [41]:
y_train.shape

(4000, 2)